<a href="https://colab.research.google.com/github/abbassix/life-simulation/blob/main/LifeSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
number_of_rows = 10
number_of_columns = 10
number_of_entities = 40
maturity_age = 20
middle_age = 50
fact_book = {}

In [3]:
def update_fact_book(entity, location=None, energy=None, creation_time=None, verbose=False):
  global fact_book
  if entity not in fact_book.keys():
    fact_book[entity] = {}
  if location:
    fact_book[entity]['location'] = location
  if creation_time:
    fact_book[entity]['creation_time'] = creation_time
  if energy:
    fact_book[entity]['energy'] += energy
  if creation_time == 0:
    fact_book[entity]['age'] = 0
    fact_book[entity]['energy'] = 1
  if verbose:
    print('\nupdate_fact_book:\n')
    print('entity, location, energy, creation_time:', entity, location, energy, creation_time)

In [4]:
def genesis(number_of_rows, number_of_columns, number_of_entities, verbose=False):
  global time_space
  time_space = np.zeros((1, number_of_rows, number_of_columns))
  entities = [n+1 for n in range(number_of_entities)]
  while len(entities):
    fiat_row = np.random.randint(number_of_rows)
    fiat_column = np.random.randint(number_of_columns)
    if time_space[0, fiat_row, fiat_column] == 0:
      update_fact_book(entity=entities[0], location=[fiat_row, fiat_column], creation_time=0)
      time_space[0, fiat_row, fiat_column] = entities[0]
      del entities[0]
  if verbose:
    print('\ngenesis:\n')
    print('time_space:', time_space, '\n')

In [5]:
def next_(time=-1, verbose=False):
  global time_space
  # create new time-layer IF there isn't a.r.
  time_space = np.append(time_space, time_space[time:,:,:], axis=0)  # create new time-layer
  if verbose:
    print('\nnext_:\n')
    print('next time:', time, '\n')

In [6]:
def aging(entity, maturity_age=maturity_age, middle_age=middle_age, verbose=False):
  global fact_book
  if fact_book[entity]['age'] <= maturity_age:
    fact_book[entity]['energy'] += 2
  elif fact_book[entity]['age'] > middle_age:
    fact_book[entity]['energy'] -= 1
  fact_book[entity]['age'] += 1
  if verbose:
    print('\naging:\n')
    print('entity, age:', entity, age, '\n')

In [7]:
genesis(number_of_rows, number_of_columns, number_of_entities, verbose=True)


genesis:

time_space: [[[33.  0.  0. 21. 25.  0.  5.  0. 29.  0.]
  [ 0. 34. 23. 27. 11.  0.  0.  0. 20.  0.]
  [ 0.  0. 35.  0.  9.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0. 37. 13.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0. 30.  7.]
  [22.  0. 15.  0.  1. 16. 28.  0.  4.  0.]
  [18.  0.  0. 10. 14.  0. 24. 19.  0.  0.]
  [32.  0. 36.  0.  2.  0.  0.  0.  0. 17.]
  [ 0. 26.  0.  0.  0.  8. 39.  0.  3. 40.]
  [ 0.  0.  6.  0. 38.  0. 12.  0.  0. 31.]]] 



In [8]:
time_space

array([[[33.,  0.,  0., 21., 25.,  0.,  5.,  0., 29.,  0.],
        [ 0., 34., 23., 27., 11.,  0.,  0.,  0., 20.,  0.],
        [ 0.,  0., 35.,  0.,  9.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., 37., 13.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 30.,  7.],
        [22.,  0., 15.,  0.,  1., 16., 28.,  0.,  4.,  0.],
        [18.,  0.,  0., 10., 14.,  0., 24., 19.,  0.,  0.],
        [32.,  0., 36.,  0.,  2.,  0.,  0.,  0.,  0., 17.],
        [ 0., 26.,  0.,  0.,  0.,  8., 39.,  0.,  3., 40.],
        [ 0.,  0.,  6.,  0., 38.,  0., 12.,  0.,  0., 31.]]])

In [9]:
def movement(entity, time, movement=None, destination=None, origin=None, verbose=False):
  global time_space
  global fact_book
  if not origin:
    origin = fact_book[entity]['location']
  if movement == 'halt':
    aging(entity)
    update_fact_book(entity=entity, location=origin)
    if verbose:
      print('\nhalt!\n')
    return
  if not destination:
    if not movement:
      raise Exception('give movement or destination')
    else:
      destination = [origin[0]+movement[0], origin[1]+movement[1]]
  if (abs(origin[0]-destination[0]) + abs(origin[1]-destination[1])) > 2 or (abs(movement[0]) + abs(movement[1])) > 2:
    raise Exception('teleportation not possible')
  elif time_space[time, origin[0], origin[1]] != entity:
    raise Exception('nothing ab initio')
  elif time_space[time+1, destination[0], destination[1]] == 0:  # <<<<<<<< time+1?? -1+1=0??
    aging(entity)
    update_fact_book(entity=entity, location=[destination[0], destination[1]])
    time_space[time+1, origin[0], origin[1]] = 0
    time_space[time+1, destination[0], destination[1]] = entity
    if verbose:
      print('\nmovement:\n')
      print('entity, time, movement, destination, origin:', entity, time, movement, destination, origin, '\n')
  else:
    raise Exception('conflict')

In [10]:
def will(entity, time=-1, verbose=False):
  global time_space
  global fact_book
  map = {}
  ip, jp = -1, -1
  print('\nwill:\n')
  if fact_book[entity]['location'][0] == 0:
    print("\nif fact_book[entity]['location'][0] == 0: ip = 0", entity, fact_book[entity]['location'][0])
    ip = 0
  if fact_book[entity]['location'][1] == 0:
    print("\nif fact_book[entity]['location'][1] == 0: jp = 0", entity, fact_book[entity]['location'][1])
    jp = 0
  ### vvvvvvvvvvvvv
  environment = time_space[-1, max(0, fact_book[entity]['location'][0]-1):min(fact_book[entity]['location'][0]+2, number_of_rows), max(0, fact_book[entity]['location'][1]-1):min(fact_book[entity]['location'][1]+2, number_of_columns)]
  from itertools import product
  nbh = lambda x, i, j: {
      (ip, jp) : x[i+ip, j+jp]
      for ip, jp in product([-1, 0, 1], repeat=2)
      if 0 <= i+ip < x.shape[0] and 0 <= j+jp < x.shape[1]
  }
  for i, j in product([0, 1, 2], repeat=2):
    if 0 <= i < environment.shape[0] and 0 <= j < environment.shape[1]:
      print('\ni, j:', i , j)
      neighbors = nbh(environment, i, j)
      map[(i+ip, j+jp)] = 0 if environment[i, j] != 0 else sum([1 for x in neighbors.values() if x == 0]) / len(neighbors)
  print('\ni+ip, j+jp, max(map, key=map.get):', i+ip, j+jp, max(map, key=map.get))
  if max(map.values()) != 0:
    movement(entity, time, movement=max(map, key=map.get), verbose=verbose)
  else:
    movement(entity, time, movement='halt', verbose=verbose)

In [11]:
def turn():
  global fact_book
  era = 50
  for t in range(era):
    next_()
    for entity in fact_book.keys():
      print('\n\nTURN:', entity)
      will(entity, time=t, verbose=True)
  

In [12]:
turn()

Streaming output truncated to the last 5000 lines.

i, j: 2 2

i+ip, j+jp, max(map, key=map.get): 1 1 (1, 0)

movement:

entity, time, movement, destination, origin: 29 45 (1, 0) [9, 2] [8, 2] 



TURN: 30

will:


i, j: 0 0

i, j: 0 1

i, j: 0 2

i, j: 1 0

i, j: 1 1

i, j: 1 2

i, j: 2 0

i, j: 2 1

i, j: 2 2

i+ip, j+jp, max(map, key=map.get): 1 1 (-1, 0)

movement:

entity, time, movement, destination, origin: 30 45 (-1, 0) [6, 2] [7, 2] 



TURN: 31

will:


i, j: 0 0

i, j: 0 1

i, j: 0 2

i, j: 1 0

i, j: 1 1

i, j: 1 2

i, j: 2 0

i, j: 2 1

i, j: 2 2

i+ip, j+jp, max(map, key=map.get): 1 1 (-1, 1)

movement:

entity, time, movement, destination, origin: 31 45 (-1, 1) [2, 8] [3, 7] 



TURN: 32

will:


if fact_book[entity]['location'][1] == 0: jp = 0 32 0

i, j: 0 0

i, j: 0 1

i, j: 1 0

i, j: 1 1

i, j: 2 0

i, j: 2 1

i+ip, j+jp, max(map, key=map.get): 1 2 (1, 0)

movement:

entity, time, movement, destination, origin: 32 45 (1, 0) [3, 0] [2, 0] 



TURN: 33

will:


i, j: 

In [13]:
time_space

array([[[33.,  0.,  0., ...,  0., 29.,  0.],
        [ 0., 34., 23., ...,  0., 20.,  0.],
        [ 0.,  0., 35., ...,  0.,  0.,  0.],
        ...,
        [32.,  0., 36., ...,  0.,  0., 17.],
        [ 0., 26.,  0., ...,  0.,  3., 40.],
        [ 0.,  0.,  6., ...,  0.,  0., 31.]],

       [[ 0., 23., 21., ...,  0.,  0.,  0.],
        [33.,  0., 35., ...,  0., 29.,  0.],
        [34.,  0.,  0., ...,  0., 20.,  0.],
        ...,
        [ 0., 18.,  0., ...,  0., 40.,  0.],
        [ 0.,  0.,  6., ...,  3., 31.,  0.],
        [26.,  0.,  0., ...,  0.,  0.,  0.]],

       [[23., 33., 35., ...,  5.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0., 29.],
        [ 0., 34.,  0., ..., 30.,  0.,  0.],
        ...,
        [ 0.,  0.,  2., ..., 24.,  0., 40.],
        [18., 26.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  6., ...,  0., 31.,  0.]],

       ...,

       [[23.,  0.,  4., ...,  0.,  6.,  0.],
        [11.,  0., 10., ...,  0., 17.,  0.],
        [ 9.,  0., 33., ...,  0., 31.,  0.

In [14]:
fact_book

{1: {'age': 50, 'energy': 43, 'location': [8, 2]},
 2: {'age': 50, 'energy': 43, 'location': [0, 4]},
 3: {'age': 50, 'energy': 43, 'location': [3, 9]},
 4: {'age': 50, 'energy': 43, 'location': [0, 0]},
 5: {'age': 50, 'energy': 43, 'location': [0, 6]},
 6: {'age': 50, 'energy': 43, 'location': [1, 8]},
 7: {'age': 50, 'energy': 43, 'location': [3, 8]},
 8: {'age': 50, 'energy': 43, 'location': [7, 8]},
 9: {'age': 50, 'energy': 43, 'location': [2, 2]},
 10: {'age': 50, 'energy': 43, 'location': [3, 0]},
 11: {'age': 50, 'energy': 43, 'location': [1, 1]},
 12: {'age': 50, 'energy': 43, 'location': [1, 4]},
 13: {'age': 50, 'energy': 43, 'location': [1, 6]},
 14: {'age': 50, 'energy': 43, 'location': [5, 5]},
 15: {'age': 50, 'energy': 43, 'location': [2, 6]},
 16: {'age': 50, 'energy': 43, 'location': [2, 4]},
 17: {'age': 50, 'energy': 43, 'location': [0, 9]},
 18: {'age': 50, 'energy': 43, 'location': [4, 5]},
 19: {'age': 50, 'energy': 43, 'location': [5, 7]},
 20: {'age': 50, 'ene

In [15]:
time_space[-1,:,:]

array([[ 4., 22., 37.,  0.,  2., 26.,  5.,  0., 31., 17.],
       [ 0., 11.,  0., 24., 12.,  0., 13.,  0.,  6., 20.],
       [23.,  0.,  9.,  0., 16.,  0., 15.,  0., 27.,  0.],
       [10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  3.],
       [ 0.,  0., 33.,  0., 40., 18.,  0., 36.,  0., 39.],
       [38.,  0., 30.,  0.,  0., 14.,  0., 19.,  0., 21.],
       [34.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [32.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.],
       [25., 35.,  1.,  0., 28.,  0., 29.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
def invasion(predator, prey, time_space=time_space):

  pass  # fight with each other!
  

In [20]:
for (x, y), z in zip(env.keys(), env.values()):
  print(x, y, z)

1 1 0.0
1 -1 0.0
1 0 17.0
-1 1 0.0
-1 -1 0.0
-1 0 0.0
0 1 14.0
0 -1 0.0
0 0 18.0


In [13]:
def look_around(entity, time=-1):
  from itertools import product
  nbh = lambda x, i, j: {
      (ip, jp) : x[i+ip, j+jp]
      for ip, jp in product([1, -1, 0], repeat=2)
      if 0 <= i+ip < x.shape[0] and 0 <= j+jp < x.shape[1]
  }
  ts = time_space[time,:,:]
  return nbh(ts, fact_book[entity]['location'][0], fact_book[entity]['location'][1])